# Deep Learning of melting curves

Here I use several ANN architectures and deep learning techniques to develop a model that can classify melting curves and compute the melting point from the shape of the curves  

In [1]:
# import all the modules/classes/functions
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, MaxPool1D, Softmax, Dropout, Input, \
    Flatten, BatchNormalization, Reshape, LeakyReLU
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import to_categorical
import numpy as np
import pandas as pd

Using TensorFlow backend.


## Prepare data

### Define some useful functions

In [2]:
# import data and labels from stored archive and encode labels
def import_data(filename):
    """imports the data/labels from a npz file and stores it into two np arrays"""
    with np.load(filename) as npzfile:
        data = npzfile["data"]
        targets = npzfile["targets"]
    
    return data, targets

# compute min/max values accross the entire training dataset
def scale_train_dataset(data):
    """data is a numpy ndarray"""
    min_val, max_val = np.min(data), np.max(data)
    scaled = (data - min_val) / (max_val - min_val)
    return (scaled, min_val, max_val)

# apply the min/max values obtained from the training set on the test set
def scale_test_dataset(data, min_val, max_val):
    """data - a numpy ndarray (test data)
    min_val, max_val - floats: min/max values obtained from the training set"""
    scaled = (data - min_val) / (max_val - min_val)
    return scaled

# reshape the targets array
def reshape_targets(targets):
    """reshapes a 1D array into a 2D array
    needed for tensorflow graphs/keras models"""
    dim = targets.shape[0]
    reshaped = targets.reshape((dim, 1))
    return reshaped
    


### Load and shuffle the dataset

In [3]:
#load augmented non-scaled data
data, targets = import_data("data_targets_aug.npz")

# shuffle the data/labels arrays consistently
data, targets = shuffle(data, targets, random_state=0)

### Split into training and test sets

Do this for both classification (prediction of curve labels) and regression (prediction of melting points). For regression, disgard the curves that belong to **class 0** (they contain too much noise and too little information to compute the melting points)

In [4]:
# split into classification and regression subsets
# classification
data_clf, targets_clf = data.copy(), targets[:,0].copy()

# regression
reg_mask = ~np.isnan(targets[:,1])
data_reg, targets_reg = data[reg_mask], targets[:,1][reg_mask]

#### Classification data

In [5]:
# split data for performance evaluation (use stratified sampling due to class imbalances)
data_clf_train, data_clf_test, labels_train, labels_test = train_test_split(data, targets_clf, test_size=0.2,
                                                                   random_state=0, stratify=targets_clf)


### Scale the data

For the GD algorithm, the curves need to be scaled to an interval of [0,1]. Since the features represent signal values at particular temperature points, I will use `min - max` scaling where the min/max values are obtained for the entire training data tensor (i.e. min/max values are for all feature values accross all curves in the training set)

#### Scale data for classification

In [6]:
# also obtain the min/max values to use on the test set
data_clf_scaled, clf_min, clf_max = scale_train_dataset(data_clf_train)
data_clf_scaled

array([[ 0.55718159,  0.55679027,  0.55621974, ...,  0.5632732 ,
         0.5668126 ,  0.57142043],
       [ 0.49458402,  0.49546699,  0.49654111, ...,  0.78559074,
         0.78784806,  0.7903608 ],
       [ 0.28658955,  0.29350023,  0.29823064, ...,  0.32790883,
         0.32816828,  0.32909247],
       ..., 
       [ 0.4850246 ,  0.48388165,  0.48328508, ...,  0.69854462,
         0.70248938,  0.70637045],
       [ 0.37028861,  0.37097712,  0.37109989, ...,  0.49555329,
         0.50682777,  0.52254469],
       [ 0.48248999,  0.50751835,  0.52853188, ...,  0.71060745,
         0.69914026,  0.68284666]])

#### Reshape data

In [7]:
labels_train = reshape_targets(labels_train)
labels_test = reshape_targets(labels_test)

## Plan

### Types of ANNs to try:

1. MLPs
2. CNNs

### CNN architectures/params to try:

- 2x 1D convolutions (#filters=16, filter_size=3, activation=ReLU)
- Batch normalization (?) (maybe I add it after each conv)
- Max pooling after each conv step (size=2)
- Flatten tensor prior to 1st FC layer
- 1 FC hidden layer (32 units to start with, activation=ReLU)
- 1 output layer (4 units, activation=Softmax)
- If overfitting, add dropout layers! (p=0.5)

### backprop params:

- loss func: categorical cross-entropy
- optm. algorithm: adam (or sgd)

## The Neural Tester class

In [8]:
# A class for building and testing ANN models

class NeuralTester(object):
    """class for testing neural net architectures"""
    def __init__(self, base_build_fn, data, targets, epochs=50, batch_size=32):
        """base_build_fn - a function object that builds the basic network (returns a keras model)
        data - rank 2 tensor of shape (n_samples, n_features)
        targets - rank 2 tensor of shape (n_samples, 1)"""
        self.base_build_fn = base_build_fn
        self.data = data
        self.targets = targets
        self.epochs = epochs
        self.batch_size = batch_size
        
    
    def fit_predict(self, build_fn, scikit_wrapper, target_name):
        """create scikit-estimator from a keras build function, fit to training, predict on test"""
        print("\nFitting the scikit estimator {}\n{}".format(scikit_wrapper.__name__, '='*40))
        build_fn().summary() # create a keras model (build the graph)
        # create the sklearn estimator (either KerasClassifier or KerasRegressor)
        estimator = scikit_wrapper(
            build_fn=build_fn, 
            epochs=self.epochs, 
            batch_size=self.batch_size, 
            verbose=1
        )
        
        # split data to train/validation sets
        data_train, data_test, targets_train, targets_test = train_test_split(self.data, self.targets,
                                                                           test_size=0.3,
                                                                           random_state=0,
                                                                           stratify=self.targets)
        
        # fit the estimator using the training set
        estimator.fit(data_train, targets_train, 
            validation_data=(data_test, targets_test), 
            callbacks=[
                ReduceLROnPlateau(patience=3, verbose=1, factor=0.1),
                EarlyStopping(patience=5, verbose=1)
            ]
        )
        
        # return predictions on test set
        return estimator.predict(data_test)
    
    def create_build_fn(self, loss, activation, optimizer='adam', metrics=[]):
        """creates the final build function by adding the output layer,
        output activitation function, loss function, and optimizer"""
        def build_fn():
            model = self.base_build_fn() # first build the base model
            model.add(Dense(1, activation=activation)) # add output layer (2 classes)
            model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
            return model
        return build_fn
 
    def run_regression(self):
        """use the build function to attempt a regression task"""
        return self.fit_predict(
            self.create_build_fn('mean_squared_error', 'linear'), 
            KerasRegressor, 
            'y_regr'
        )
        
    
    def run_classification(self):
        """use the build function to attempt a classification task"""
        return self.fit_predict(
            self.create_build_fn('binary_crossentropy', 'softmax', metrics=['accuracy']), 
            KerasClassifier, 
            'y_cls'
        )



## Build and train a simple MLP

In [13]:
def create_model_FC():
    """creates a simple MLP"""
    model = Sequential()
    model.add(Dense(16, input_dim=100, activation="relu"))
    model.add(Dense(16, activation="sigmoid"))
    return model


In [14]:
# instantiate the test class and train a MPL
tester_fc = NeuralTester(create_model_FC, data_clf_train, labels_train, epochs=10, batch_size=100)
clss_fc = tester_fc.run_classification()


Fitting the scikit estimator KerasClassifier
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 16)                1616      
_________________________________________________________________
dense_14 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 17        
Total params: 1,905
Trainable params: 1,905
Non-trainable params: 0
_________________________________________________________________
Train on 8400 samples, validate on 3600 samples
Epoch 1/10
8400/8400 [==============================] - 1s 62us/step - loss: 6.6047 - acc: 0.5857 - val_loss: 6.6072 - val_acc: 0.5856
Epoch 2/10
8400/8400 [==============================] - 0s 37us/step - loss: 6.6047 - acc: 0.5857 - val_loss: 6.6072 - val_acc: 0.5856
Epoch 3/10
8400/8400 [==================

## Build and train a 1D CNN

### Reshape data

In [9]:
# reshape data into a rank 3 tensor: (n_samples, 100, 1). Required by 1D conv layer
data_clf_scaled_cnn = data_clf_scaled.reshape(data_clf_scaled.shape[0], data_clf_scaled.shape[1], 1)
print(data_clf_scaled_cnn.shape)

(12000, 100, 1)


In [10]:
def create_model_1Dconv():
    """creates a start model to be completed by the NeuralTester class"""
    model = Sequential()
    
    model.add(Conv1D(4, 7, strides=2, input_shape=(100, 1), activation="relu"))
    #model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.3))
    #model.add(Conv1D(16, 3, activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    
    model.add(Conv1D(4, 5, strides=1, activation="relu"))
    #model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.3))
    #model.add(Conv1D(16, 3, activation='relu'))
    model.add(MaxPool1D(pool_size=2))
    
    #model.add(Conv1D(16, 5))
    #model.add(BatchNormalization())
    #model.add(LeakyReLU(alpha=0.3))
    #model.add(Conv1D(16, 3, activation='relu'))
    #model.add(MaxPool1D(pool_size=2))
    
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    return model


### Train a Conv. Net

In [ ]:
# Instantiate our testing class and train
tester = NeuralTester(create_model_1Dconv,
                      data_clf_scaled_cnn,
                      labels_train,
                      epochs=10,
                      batch_size=100)

clss = tester.run_classification()


Fitting the scikit estimator KerasClassifier
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 47, 4)             32        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 23, 4)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 19, 4)             84        
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 9, 4)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2368      
_________________________________________________________________
dense_2 (Dense)              (

# Tune the hyperparameters of the neural net

These parameters could be:
-  filter size for conv nets (int)
-  num. of filters for each conv layer (int)
-  num. of units in the FC layer (int)
-  dropout rate (float)
-  etc.

One could use either a grid search or a Baysean optimization process to find an optimal set of hyperparameters. In each case a new net is constructed and evaluated using CV on the validation set. The best net is then evaluated against a final test set which was set aside in the beginning